In [46]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import arff
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.impute import KNNImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score, auc, roc_curve
from sklearn.metrics import make_scorer
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.decomposition import PCA
import os

In [47]:
path_to_directory="ReLink/"
files = [arff for arff in os.listdir(path_to_directory) if arff.endswith(".arff")]

def toCsv(content): 
    data = False
    header = ""
    newContent = []
    for line in content:
        if not data:
            if "@attribute" in line:
                attri = line.split()
                columnName = attri[attri.index("@attribute")+1]
                header = header + columnName + ","
            elif "@data" in line:
                data = True
                header = header[:-1]
                header += '\n'
                newContent.append(header)
        else:
            newContent.append(line)
    return newContent

In [48]:
# Main loop for reading and writing files
for zzzz,file in enumerate(files):
    with open(path_to_directory+file , "r") as inFile:
        content = inFile.readlines()
        name,ext = os.path.splitext(inFile.name)
        new = toCsv(content)
        with open(name+".csv", "w") as outFile:
            outFile.writelines(new)

In [49]:
apache = pd.read_csv('Relink/Apache.csv')
safe = pd.read_csv('Relink/Safe.csv')
zxing = pd.read_csv('Relink/Zxing.csv')

In [50]:
list_relink = ['apache','safe','zxing']
list_data_relink = [apache,safe,zxing]

In [51]:
zxing

,AvgCyclomatic,AvgCyclomaticModified,AvgCyclomaticStrict,AvgEssential,AvgLine,AvgLineBlank,AvgLineCode,AvgLineComment,CountLine,CountLineBlank,...,CountStmtExe,MaxCyclomatic,MaxCyclomaticModified,MaxCyclomaticStrict,RatioCommentToCode,SumCyclomatic,SumCyclomaticModified,SumCyclomaticStrict,SumEssential,isDefective
0,### AcgCy AvgCyMod AvgCyStrict AvgEssential Av...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2.0,2.0,1.0,7.0,0.0,7.0,1.0,66.0,6.0,...,17.0,4.0,4.0,5.0,0.80,9.0,9.0,10.0,4.0,False
2,1,1.0,1.0,1.0,7.0,0.0,6.0,1.0,17.0,2.0,...,2.0,1.0,1.0,1.0,0.15,2.0,2.0,2.0,2.0,True
3,1,1.0,1.0,1.0,6.0,1.0,5.0,0.0,75.0,20.0,...,9.0,1.0,1.0,1.0,0.00,9.0,9.0,9.0,9.0,False
4,1,1.0,1.0,1.0,10.0,0.0,10.0,1.0,176.0,17.0,...,86.0,4.0,4.0,4.0,0.38,20.0,20.0,20.0,12.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,1,1.0,1.0,1.0,10.0,0.0,10.0,0.0,42.0,5.0,...,11.0,2.0,2.0,2.0,0.23,3.0,3.0,3.0,2.0,False
396,1,1.0,2.0,1.0,24.0,4.0,17.0,3.0,63.0,8.0,...,22.0,1.0,1.0,2.0,0.17,1.0,1.0,2.0,1.0,False
397,1,1.0,1.0,1.0,13.0,1.0,9.0,2.0,96.0,13.0,...,20.0,3.0,3.0,3.0,0.98,7.0,7.0,7.0,4.0,False
398,1,1.0,2.0,1.0,7.0,0.0,7.0,0.0,23.0,5.0,...,5.0,2.0,2.0,3.0,0.00,3.0,3.0,4.0,2.0,False


In [52]:
for dt in list_data_relink:
    dt.drop(index=dt.index[0], axis=0, inplace=True)
    dt['Defective'] = dt['isDefective']
    dt.loc[dt['Defective'] == False, 'Defective'] = 'N'
    dt.loc[dt['Defective'] == True, 'Defective'] = 'Y'
    dt.drop('isDefective',axis=1,inplace=True)

In [53]:
list_dataset = [apache,zxing]
label_dataset = ['apache','safe']

list_test = [apache,safe,zxing]
label_test = ['apache','safe','zxing']

my_index = range(0,len(list_test))
my_dataset = dict(zip(label_test,list_test))

df = list_dataset
df = pd.concat(df)
df

,AvgCyclomatic,AvgCyclomaticModified,AvgCyclomaticStrict,AvgEssential,AvgLine,AvgLineBlank,AvgLineCode,AvgLineComment,CountLine,CountLineBlank,...,CountStmtExe,MaxCyclomatic,MaxCyclomaticModified,MaxCyclomaticStrict,RatioCommentToCode,SumCyclomatic,SumCyclomaticModified,SumCyclomaticStrict,SumEssential,Defective
1,12,11.0,14.0,4.0,82.0,5.0,61.0,7.0,2195.0,165.0,...,738.0,51.0,51.0,56.0,0.28,221.0,197.0,246.0,81.0,Y
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,644.0,1.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,N
3,8,7.0,9.0,4.0,54.0,4.0,49.0,1.0,1945.0,209.0,...,877.0,45.0,26.0,45.0,0.04,256.0,217.0,277.0,128.0,Y
4,5,5.0,6.0,2.0,37.0,5.0,27.0,5.0,1159.0,171.0,...,414.0,25.0,25.0,30.0,0.32,134.0,122.0,146.0,63.0,N
5,2,2.0,2.0,1.0,21.0,1.0,9.0,3.0,166.0,15.0,...,20.0,4.0,5.0,4.0,0.73,9.0,10.0,9.0,10.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,1,1.0,1.0,1.0,10.0,0.0,10.0,0.0,42.0,5.0,...,11.0,2.0,2.0,2.0,0.23,3.0,3.0,3.0,2.0,N
396,1,1.0,2.0,1.0,24.0,4.0,17.0,3.0,63.0,8.0,...,22.0,1.0,1.0,2.0,0.17,1.0,1.0,2.0,1.0,N
397,1,1.0,1.0,1.0,13.0,1.0,9.0,2.0,96.0,13.0,...,20.0,3.0,3.0,3.0,0.98,7.0,7.0,7.0,4.0,N
398,1,1.0,2.0,1.0,7.0,0.0,7.0,0.0,23.0,5.0,...,5.0,2.0,2.0,3.0,0.00,3.0,3.0,4.0,2.0,N


In [54]:
def adjustdataset_list(list_df):
    dt_result = list_df.pop()
    
    for dt in list_df:
        if len(dt.columns) > len(dt_result.columns):
            dt_result = adjustdataset(dt,dt_result)
        else:
            dt_result = adjustdataset(dt_result,dt)    
    return dt_result

In [55]:
def adjustdataset(dt,dataframe):
    dt_copy = dt.copy()
    counter = 0
    
    for col in dataframe.columns:
        if not col in dt.columns:
            #print("Not Exist ",col)
            dt_copy.insert(counter,col,-1)
        counter += 1

    return pd.concat([dt_copy[dataframe.columns], dataframe])
    

In [56]:
def show_distribution(df, columns):
    
    counter = 0
    # Create a figure and a grid of subplots with 4 rows and 10 columns
    fig, axes = plt.subplots(5, 8, figsize=(20, 8))
    
    # Loop through each subplot and add content (optional)
    for i in range(5):
        for j in range(8):
            try:
                ax = axes[i, j]
                ax.set_title(columns[counter])
                ax.plot(df[columns[counter]].sort_values(), norm.pdf(df[columns[counter]].sort_values(), df[columns[counter]].mean(), df[columns[counter]].std())) 
                ax.set_xticks([])
                ax.set_yticks([])
                counter += 1
            except:
                pass

# Adjust layout to prevent overlapping titles/labels
plt.tight_layout()

# Display the plot
plt.show()

<Figure size 640x480 with 0 Axes>

In [57]:
def preprocessing(dataframe, imputer = None, pt = None, scaler = None, show = True):
    
    df = dataframe.copy()
    
    df.loc[(df['Defective'] == 'N'),'Defective']=0
    df.loc[(df['Defective'] == 'Y'),'Defective']=1
    
    change_dtypes = ['Defective']
    dict_dtypes = {d:'int64' for d in change_dtypes }
    df = df.astype(dict_dtypes)
    
    columns = df.columns
    
    if show:
        show_distribution(df,df.columns)
    
    if imputer :
        df = imputer.transform(df)
    else:
        imputer = KNNImputer(n_neighbors=2)
        df = imputer.fit_transform(df)
    
    df = pd.DataFrame(df)
    df.columns = columns
    
    if show:
        show_distribution(df,df.columns)
    
    y = df['Defective']
    X = df.drop('Defective',axis=1)
    
    if pt:
        X_norm = pt.transform(X)
    else:
        pt = PowerTransformer()
        X_norm = pt.fit_transform(X)
    
    X_norm = pd.DataFrame(X)
    X_norm.columns = X.columns
    
    if show:
        show_distribution(X_norm,X_norm.columns)

    if scaler:
        X_norm_scale = scaler.transform(X_norm)
    else:
        scaler = StandardScaler()
        X_norm_scale = scaler.fit_transform(X_norm)
        
    return pt, scaler, imputer, X_norm_scale, y



In [58]:
def heat_map(y,result,title):
    import seaborn as sns
    from sklearn.metrics import confusion_matrix
    sns.set
    _, ax = plt.subplots(figsize=(10,10))
    ax = sns.heatmap(confusion_matrix(y, result), annot=True, fmt='d', cmap= ['#09acec','#089dd9','#078fc5','#0681b1','#05668d','#05368d'], annot_kws={"size": 40, "weight": "bold"})  
    labels = ['No Defect','Defect']
    ax.set_xticklabels(labels, fontsize=30);
    ax.set_yticklabels(labels, fontsize=30,rotation=0);
    ax.set_ylabel('Prediction', fontsize=30,rotation=0);
    ax.set_xlabel('Ground Truth', fontsize=30) #0,1
    ax.set_title(title)

In [59]:
def score(y,result):
    accuracy = accuracy_score(y, result)
    f1 = f1_score(y, result, average='weighted')
    precision = precision_score(y, result, average='weighted')
    recall = recall_score(y, result, average='weighted')
    print("Accuracy = " , accuracy, " Precision = ",precision, " Recall = ",recall, " F1-Score = ",f1 )

In [60]:
from sklearn.metrics import make_scorer

scorers = {
            'f1_score': make_scorer(f1_score, average='micro'),
            'precision_score': make_scorer(precision_score, average='micro'),
            'recall_score': make_scorer(recall_score, average='micro'),
            'accuracy_score': make_scorer(accuracy_score)
          }
search_space={}

In [61]:
pt, scaler, imputer, X_norm_scale, y = preprocessing(df, show=False)
#X_norm_scale_smote, y_smote = X_norm_scale, y
smote = SMOTE(random_state=42)
X_norm_scale_smote, y_smote = smote.fit_resample(X_norm_scale, y)  

/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


In [62]:
counter = 0
histories = []

lst_accu_stratified = []
lst_f1_stratified = []
lst_precision_stratified = []
lst_recall_stratified = []
lst_auc_stratified = []

lst_accu_stratified_val = []
lst_f1_stratified_val = []
lst_precision_stratified_val = []
lst_recall_stratified_val = []
lst_auc_stratified_val = []

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=19)

for train_index, test_index in skf.split(X_norm_scale_smote, y_smote):

    x_train_fold, x_test_fold = X_norm_scale_smote[train_index], X_norm_scale_smote[test_index]
    y_train_fold, y_test_fold = y_smote[train_index], y_smote[test_index]
 
    rte = RandomTreesEmbedding(n_estimators=1000, random_state=0, max_depth=10).fit(x_train_fold)
    x_sparse_embedding_train = rte.transform(x_train_fold)
    x_sparse_embedding_test = rte.transform(x_test_fold)
    
    with open('rte-tree-relink-{0}.pickle'.format(counter), 'wb') as handle:
        pickle.dump(rte, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    from sklearn.manifold import Isomap
    manifold = Isomap(n_components=100)
    x_sparse_manifold_train = manifold.fit_transform(x_sparse_embedding_train)
    x_sparse_manifold_test = manifold.transform(x_sparse_embedding_test)
    
    with open('manifold-tree-relink-{0}.pickle'.format(counter), 'wb') as handle:
        pickle.dump(manifold, handle, protocol=pickle.HIGHEST_PROTOCOL)

    scaler2 = StandardScaler()
    x_sparse_manifold_train = scaler2.fit_transform(x_sparse_manifold_train)
    x_sparse_manifold_test = scaler2.transform(x_sparse_manifold_test)
    
    with open('scaler-tree-relink-{0}.pickle'.format(counter), 'wb') as handle:
        pickle.dump(scaler2, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    input_X_train1 = layers.Input(
    shape=(x_sparse_manifold_train.shape[1],), name="X_train1")  
    
    input_X_train2 = layers.Input(shape=(x_train_fold.shape[1],), name="X_train2")

    layer_X_train1 = layers.Dense(64,activation='relu',name="X_train1_layer")(input_X_train1)
    layer_X_train1 = layers.BatchNormalization()(layer_X_train1)
    layer_X_train1 = layers.Dropout(0.3)(layer_X_train1)
    
    layer_X_train2 = layers.Dense(64,activation='relu',name="X_train2_layer")(input_X_train2)
    layer_X_train2 = layers.BatchNormalization()(layer_X_train2)
    layer_X_train2 = layers.Dropout(0.3)(layer_X_train2)
    
    # Merge all available features into a single large vector via concatenation
    concat = layers.concatenate([layer_X_train1, layer_X_train2])

    layer_final = layers.Dense(1, activation='sigmoid', name="classifier")(concat)
    sdp_classifier = Model(inputs=[input_X_train1, input_X_train2], outputs=layer_final)

    checkpointer=ModelCheckpoint(filepath='sdp-tree-relink-{0}.weights.best.keras'.format(counter),verbose=1,save_best_only=True)

    sdp_classifier.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'] )
    
    hist = sdp_classifier.fit([x_sparse_manifold_train,x_train_fold], y_train_fold, 
                    batch_size=32, epochs=100, validation_split= 0.1, callbacks=[checkpointer], 
                    verbose=2, shuffle=True)
    
    with open('sdp-classifier-tree-relink-{0}.pickle'.format(counter), 'wb') as handle:
        pickle.dump(sdp_classifier, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

    sdp_classifier.load_weights('sdp-tree-relink-{0}.weights.best.keras'.format(counter))

    histories.append(hist)

    results = sdp_classifier.predict([x_sparse_manifold_train,x_train_fold])
    results = np.round(results)
    
    lst_accu_stratified.append(accuracy_score(y_train_fold, results))
    lst_f1_stratified.append(f1_score(y_train_fold, results, average='weighted') )
    lst_precision_stratified.append(precision_score(results, y_train_fold, average='weighted') )
    lst_recall_stratified.append(recall_score(results, y_train_fold, average='weighted') )
    fpr, tpr, thresholds = roc_curve(y_train_fold, results)
    lst_auc_stratified.append(auc(fpr, tpr))
    
    results = sdp_classifier.predict([x_sparse_manifold_test,x_test_fold])
    results = np.round(results)
    
    lst_accu_stratified_val.append(accuracy_score(y_test_fold, results))
    lst_f1_stratified_val.append(f1_score(results, y_test_fold, average='weighted') )
    lst_precision_stratified_val.append(precision_score(results, y_test_fold, average='weighted') )
    lst_recall_stratified_val.append(recall_score(results, y_test_fold, average='weighted') )
    fpr, tpr, thresholds = roc_curve(y_test_fold, results)
    lst_auc_stratified_val.append(auc(fpr, tpr))

    print("Evaluate on training data")
    results = sdp_classifier.evaluate([x_sparse_manifold_train, x_train_fold], y_train_fold)
    print("test loss, test acc:", results)

    print("Evaluate on test data")
    results = sdp_classifier.evaluate([x_sparse_manifold_test , x_test_fold], y_test_fold)
    print("test loss, test acc:", results)
    counter += 1
    

dict_test = {  "train_accu":lst_accu_stratified, 
               "train_precision":lst_precision_stratified, "train_recall":lst_recall_stratified,
               "train_auc":lst_auc_stratified, "train_f1":lst_f1_stratified,
             
               "test_accu":lst_accu_stratified_val, 
               "test_precision":lst_precision_stratified_val, "test_recall":lst_recall_stratified_val, 
               "test_auc":lst_auc_stratified_val, "test_f1":lst_f1_stratified_val,
               "Input": x_train_fold.shape[1], "RTE": x_sparse_manifold_train.shape[1]}

dict_test_result = pd.DataFrame.from_dict(dict_test)
dict_test_result.to_csv("hasil.csv")

with open('histories-tree-manifold-relink.pickle'.format(counter), 'wb') as handle:
        pickle.dump(histories, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

Epoch 1/100

Epoch 1: val_loss improved from inf to 0.52614, saving model to sdp-tree-relink-0.weights.best.keras
20/20 - 2s - loss: 0.8629 - accuracy: 0.5492 - val_loss: 0.5261 - val_accuracy: 0.7500 - 2s/epoch - 86ms/step
Epoch 2/100

Epoch 2: val_loss did not improve from 0.52614
20/20 - 0s - loss: 0.6975 - accuracy: 0.6508 - val_loss: 0.5694 - val_accuracy: 0.6471 - 64ms/epoch - 3ms/step
Epoch 3/100

Epoch 3: val_loss did not improve from 0.52614
20/20 - 0s - loss: 0.6777 - accuracy: 0.6721 - val_loss: 0.6277 - val_accuracy: 0.5441 - 64ms/epoch - 3ms/step
Epoch 4/100

Epoch 4: val_loss did not improve from 0.52614
20/20 - 0s - loss: 0.5837 - accuracy: 0.6951 - val_loss: 0.6548 - val_accuracy: 0.5000 - 67ms/epoch - 3ms/step
Epoch 5/100

Epoch 5: val_loss did not improve from 0.52614
20/20 - 0s - loss: 0.5764 - accuracy: 0.6836 - val_loss: 0.6602 - val_accuracy: 0.5000 - 57ms/epoch - 3ms/step
Epoch 6/100

Epoch 6: val_loss did not improve from 0.52614
20/20 - 0s - loss: 0.5054 - accu

/opt/anaconda3/lib/python3.11/site-packages/sklearn/manifold/_isomap.py:384: UserWarning: The number of connected components of the neighbors graph is 2 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
/opt/anaconda3/lib/python3.11/site-packages/scipy/sparse/_index.py:100: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


Epoch 1/100

Epoch 1: val_loss improved from inf to 0.77514, saving model to sdp-tree-relink-1.weights.best.keras
20/20 - 2s - loss: 0.8848 - accuracy: 0.5836 - val_loss: 0.7751 - val_accuracy: 0.3971 - 2s/epoch - 76ms/step
Epoch 2/100

Epoch 2: val_loss did not improve from 0.77514
20/20 - 0s - loss: 0.6969 - accuracy: 0.6164 - val_loss: 0.7878 - val_accuracy: 0.3971 - 63ms/epoch - 3ms/step
Epoch 3/100

Epoch 3: val_loss did not improve from 0.77514
20/20 - 0s - loss: 0.6122 - accuracy: 0.6754 - val_loss: 0.8310 - val_accuracy: 0.3529 - 65ms/epoch - 3ms/step
Epoch 4/100

Epoch 4: val_loss did not improve from 0.77514
20/20 - 0s - loss: 0.5877 - accuracy: 0.7066 - val_loss: 0.8000 - val_accuracy: 0.3824 - 65ms/epoch - 3ms/step
Epoch 5/100

Epoch 5: val_loss improved from 0.77514 to 0.75122, saving model to sdp-tree-relink-1.weights.best.keras
20/20 - 0s - loss: 0.5721 - accuracy: 0.7115 - val_loss: 0.7512 - val_accuracy: 0.4853 - 110ms/epoch - 5ms/step
Epoch 6/100

Epoch 6: val_loss im

In [63]:
dict_test_result

,train_accu,train_precision,train_recall,train_auc,train_f1,test_accu,test_precision,test_recall,test_auc,test_f1,Input,RTE
0,0.594395,0.651935,0.594395,0.594395,0.582380,0.605263,0.693906,0.605263,0.605263,0.623570,26,100
1,0.967552,0.968178,0.967552,0.967552,0.967541,0.631579,0.631579,0.631579,0.631579,0.631579,26,100
2,0.954277,0.955013,0.954277,0.954277,0.954260,0.697368,0.697715,0.697368,0.697368,0.697421,26,100
3,0.961652,0.962278,0.961652,0.961652,0.961640,0.657895,0.663435,0.657895,0.657895,0.658845,26,100
4,0.951399,0.952656,0.951399,0.951362,0.951367,0.773333,0.815477,0.773333,0.775249,0.777846,26,100
5,0.923417,0.923854,0.923417,0.923395,0.923399,0.666667,0.666904,0.666667,0.666785,0.666667,26,100
6,0.917526,0.918155,0.917526,0.917500,0.917498,0.773333,0.776776,0.773333,0.772760,0.773818,26,100
7,0.951399,0.951749,0.951399,0.951419,0.951392,0.720000,0.722902,0.720000,0.720484,0.720299,26,100
8,0.949926,0.952021,0.949926,0.949974,0.949876,0.640000,0.656529,0.640000,0.641181,0.642705,26,100
9,0.942563,0.943535,0.942563,0.942595,0.942536,0.680000,0.686145,0.680000,0.679232,0.681143,26,100


In [76]:
best = 1

dbfile = open('rte-tree-relink-{0}.pickle'.format(best), 'rb')    
rte = pickle.load(dbfile)
dbfile.close()

dbfile = open('manifold-tree-relink-{0}.pickle'.format(best), 'rb')    
manifold = pickle.load(dbfile)
dbfile.close()

dbfile = open('scaler-tree-relink-{0}.pickle'.format(best), 'rb')    
scaler2 = pickle.load(dbfile)
dbfile.close()

dbfile = open('sdp-classifier-tree-relink-{0}.pickle'.format(best), 'rb')    
sdp_classifier = pickle.load(dbfile)
dbfile.close()
sdp_classifier.load_weights('sdp-tree-relink-{0}.weights.best.keras'.format(best))

In [77]:
X_sparse_embedding = rte.transform(X_norm_scale)
X_sparse_manifold = manifold.transform(X_sparse_embedding)
X_sparse_manifold = scaler2.transform(X_sparse_manifold)
X_sparse_embedding.shape

(593, 85053)

In [78]:
X_norm_scale.shape, df.shape

((593, 26), (593, 27))

In [79]:
print("Evaluate on training data")
results = sdp_classifier.evaluate([X_sparse_manifold, X_norm_scale], y, batch_size=128)
print("test loss, test acc:", results)

Evaluate on training data
5/5 [==============================] - 0s 2ms/step - loss: 0.1623 - accuracy: 0.9444
test loss, test acc: [0.16230954229831696, 0.9443507790565491]


In [80]:
from sklearn.metrics import classification_report

results = sdp_classifier.predict([X_sparse_manifold,X_norm_scale])
results=np.round(results)
print(classification_report(y,results))

19/19 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

         0.0       0.96      0.95      0.96       377
         1.0       0.91      0.94      0.92       216

    accuracy                           0.94       593
   macro avg       0.94      0.94      0.94       593
weighted avg       0.94      0.94      0.94       593



In [81]:
print(y.shape, results.shape)
fpr, tpr, thresholds = roc_curve(y, results)
print(auc(fpr, tpr))

(593,) (593, 1)
0.94239365360055


In [82]:
list_result = []

for label in my_dataset:
    dt = adjustdataset(my_dataset[label], df)
    pt, scaler, imputer, X_norm_scale_dt, y_dt = preprocessing(dt,imputer,pt, scaler, show=False)
    x_sparse_embedding_dt = rte.transform(X_norm_scale_dt)
    x_sparse_manifold_dt = manifold.transform(x_sparse_embedding_dt)
    x_sparse_manifold_dt = scaler2.transform(x_sparse_manifold_dt)
    results = sdp_classifier.predict([x_sparse_manifold_dt,X_norm_scale_dt])
    result  = np.round(results)
    fpr, tpr, thresholds = roc_curve(y_dt, result)
    list_result.append({'label':label, 'accuracy' : accuracy_score(y_dt, result),'precision':precision_score(y_dt, result, average='weighted'),
                  'recall': recall_score(y_dt, result, average='weighted'), 'auc':auc(fpr, tpr),
                        'f1-score':f1_score(y_dt, result, average='weighted')})
    #heat_map(y_dt,result,label)


    

31/31 [==============================] - 0s 1ms/step


In [83]:
df_result = pd.DataFrame(list_result)
df_result

,label,accuracy,precision,recall,auc,f1-score
0,apache,0.941550,0.941843,0.941550,0.940669,0.941640
1,safe,0.916795,0.917155,0.916795,0.912199,0.916938
2,zxing,0.946573,0.947085,0.946573,0.943510,0.946744


In [72]:
df_result[['accuracy','precision','recall','auc','f1-score']].mean()

accuracy     0.554275
precision    0.648666
recall       0.554275
auc          0.600949
f1-score     0.552096
dtype: float64

In [73]:
sdp_classifier.summary()

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 X_train1 (InputLayer)       [(None, 100)]                0         []                            
                                                                                                  
 X_train2 (InputLayer)       [(None, 26)]                 0         []                            
                                                                                                  
 X_train1_layer (Dense)      (None, 64)                   6464      ['X_train1[0][0]']            
                                                                                                  
 X_train2_layer (Dense)      (None, 64)                   1728      ['X_train2[0][0]']            
                                                                                           

In [74]:
list_result = []

for label in my_dataset:
    dt = my_dataset[label]
    list_result.append({'Dataset':label, 'Instances':dt.shape[0], 'Features':dt.shape[1],
                        'Defective Instances': dt[dt['Defective'] == 'Y'].shape[0],
                        'Non-Defective Instances': dt[dt['Defective'] == 'N'].shape[0],})


    

In [75]:
df_result = pd.DataFrame(list_result)
df_result

,Dataset,Instances,Features,Defective Instances,Non-Defective Instances
0,apache,194,27,98,96
1,safe,56,27,22,34
2,zxing,399,27,118,281
